# Load files

In [1]:
import pandas as pd
import numpy as np
from algorithms import load_data
data_diam = load_data('data.csv','mood')
print(data_diam[['mood', 'water']][0:2])

  mood  water
0    4    2.0
1    5    1.0


# Ordinal logistic regression - playground

test logistic regression on derived modd and basic amount sleep phiscal activity and water

The method determines which solver from scipy.optimize is used, and it can be chosen from among the following strings:

*    ‘newton’ for Newton-Raphson, ‘nm’ for Nelder-Mead

*    ‘bfgs’ for Broyden-Fletcher-Goldfarb-Shanno (BFGS)

*   ‘lbfgs’ for limited-memory BFGS with optional box constraints

*    ‘powell’ for modified Powell’s method

*    ‘cg’ for conjugate gradient

*   ‘ncg’ for Newton-conjugate gradient

*   ‘basinhopping’ for global basin-hopping solver

*   ‘minimize’ for generic wrapper of scipy minimize (BFGS by default)


In [2]:
from statsmodels.miscmodels.ordinal_model import OrderedModel
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

derived = 'mood'
lbasics = ['amount of sleep',  'physical activity', 'water']
from algorithms import generateOLR
res_log = generateOLR(data_diam, derived, lbasics)
print(res_log.summary())

                             OrderedModel Results                             
Dep. Variable:                   mood   Log-Likelihood:            -1.1331e-07
Model:                   OrderedModel   AIC:                             14.00
Method:            Maximum Likelihood   BIC:                             12.54
Date:                Wed, 18 May 2022                                         
Time:                        12:35:28                                         
No. Observations:                   6                                         
Df Residuals:                      -1                                         
Df Model:                           7                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
amount of sleep     435.0279        nan        nan        nan         nan         nan
physical activity  -397.5140   

c:\Users\do2vl\anaconda3\lib\site-packages\statsmodels\base\model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


# check t value to p value

In [3]:
from algorithms import ttest 
res = ttest(res_log, data_diam, 'mood', ['amount of sleep',  'physical activity', 'water'])
print(f'res/pvalue: {res}')

res/pvalue: 0.2665662286967403


# pred

In [4]:
pred_row = np.array(
    [[0.1, 0.2, 0.2, 0.1 , 0.4],
    [0.4, 0.4, 0.1, 0.04, 0.06],
    ]
)
print(pred_row)
print(pred_row.shape)
max_val = pred_row[:].max(1).reshape((-1,1))
# print(max_val)
(pred_i , pred_j) = np.where(pred_row == max_val) # get list indexe
pred = list(map(lambda x: pred_j[x], set(pred_i)))
print(pred)

index_vec = np.array([1,2,3,4,5])
pred = [np.sum(np.multiply(index_vec,pred_row[i][:])).round() for i in range(pred_row.shape[0])]
print(pred)

[[0.1  0.2  0.2  0.1  0.4 ]
 [0.4  0.4  0.1  0.04 0.06]]
(2, 5)
[4, 0]
[4.0, 2.0]


# Stepwise

In [5]:
import numpy as np
from algorithms import generateOLR
from algorithms import ttest
from copy import copy

import warnings
warnings.filterwarnings("ignore") # ignore warning

print(data_diam)

data = data_diam
in_threshold = 0.3
out_threshold = 0.2
sderived = 'mood'
lbasics = list(data.columns.values)
lbasics.remove(sderived)
from algorithms import stepwise_regression
(fit_model, param_in_model) = stepwise_regression(data, sderived, lbasics, in_threshold, out_threshold)
print(f'param_in_model: {param_in_model}')


  mood  amount of sleep  physical activity  cognative load  water  \
0    4              9.0                5.0             2.0    2.0   
1    5              6.0                1.0             3.0    1.0   
2    4             10.0                4.0             2.0    2.0   
3    1              8.0                4.0             5.0    4.0   
4    2              7.0                1.0             3.0    4.0   
5    3              8.0                2.0             2.0    4.0   

   quality of sleep  
0               5.0  
1               3.0  
2               4.0  
3               2.0  
4               2.0  
5               3.0  
add-params: ['cognative load'], pvalue: 0.13062572116925905
add-params: ['cognative load', 'physical activity'], pvalue: 0.13062572116925905
add-params: ['cognative load', 'physical activity', 'amount of sleep'], pvalue: 0.2107520298688622
add-params: ['cognative load', 'physical activity', 'amount of sleep', 'water'], pvalue: 0.2665662286967403
add-params: ['

# Acurcay and distance vector

In [6]:
import pandas as pd
import numpy as np

from evaluation import distance_vec
data = data_diam
sderived = 'mood'
lbasics = param_in_model
distance = distance_vec(data, sderived, lbasics, fit_model)


distance['dist'][2] = 2
print(distance)
from evaluation import accuracy
print(f'accuracy: {accuracy(distance)}')

   derived  pred  dist
0        4     3     1
1        5     4     1
2        4     3     2
3        1     0     1
4        2     1     1
5        3     2     1
accuracy: 7


# Filters

In [9]:
from data_manipulation import avg_filter, filter

data = [ 1,2,3,4,5,6]
filter_size = 3
func = avg_filter(filter_size)
res = filter(data, func, filter_size)
print(res)

# filter before
import pandas as pd
data_diam = pd.read_csv("data.csv", dtype=np.double)
print(data_diam)
lparams = list(data_diam.columns.values)

size_sample = data_diam.shape[0]



data = np.empty((size_sample//filter_size, len(lparams)), dtype=np.double)
for i, param in enumerate(lparams):
    data[:,i] = filter(data_diam[param].values, func,filter_size)
    

#df = pd.DataFrame(data, columns=lparams)
#print(df)

lfilters = [ func ] * len(lparams)
from algorithms import load_data_with_fiters
sderived = "amount of sleep"
df = load_data_with_fiters("30days.csv", sderived, lfilters, filter_size)
print(df[0:5])



[2.0, 5.0]
   mood  amount of sleep  physical activity  cognative load  water  \
0   4.0              9.0                5.0             2.0    2.0   
1   5.0              6.0                1.0             3.0    1.0   
2   4.0             10.0                4.0             2.0    2.0   
3   1.0              8.0                4.0             5.0    4.0   
4   2.0              7.0                1.0             3.0    4.0   
5   3.0              8.0                2.0             2.0    4.0   

   quality of sleep  
0               5.0  
1               3.0  
2               4.0  
3               2.0  
4               2.0  
5               3.0  
       mood amount of sleep  physical activity  cognitive load     water  \
0  3.666667               4           1.666667       -0.333333  1.666667   
1  1.000000               5          -0.333333        1.666667  4.000000   
2  3.000000               1          -1.000000        3.333333  3.666667   
3  2.333333               2           0.